In [ ]:
import nest_asyncio
import asyncio
import logging
from telethon import TelegramClient, events
from openai import OpenAI

# تفعيل nest_asyncio لتجنب تعارض الحلقات التزامنية في Jupyter أو Colab
nest_asyncio.apply()

# تعريف المعلومات اللازمة
api_id = "21403141"
api_hash = "74e9c96971cd422c1ba9878a13ea3db1"
bot_token = "7847748804:AAGrnFtZLY9GH6K1Rk0Mo_fX5V0bLt0dcr4"

# تهيئة العميل
client = TelegramClient('bot', api_id, api_hash)

# تهيئة OpenRouter باستخدام API Key الصحيح
openrouter_client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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"
)

# تعريف الدالة الرئيسية
async def main():
    # بدء العميل
    await client.start(bot_token=bot_token)

    # معالج الأمر /start
    @client.on(events.NewMessage(pattern='/start'))
    async def start_handler(event):
        await event.respond("Hello! I am Hackers Realm AI Bot. How can I assist you today?")
        logging.info(f'Start command received from {event.sender_id}')

    # معالج الأمر /info
    @client.on(events.NewMessage(pattern='/info'))
    async def info_handler(event):
        await event.respond("This AI Chatbot is created using OpenRouter.")
        logging.info(f'Info command received from {event.sender_id}')

    # معالج الأمر /help
    @client.on(events.NewMessage(pattern='/help'))
    async def help_handler(event):
        help_text = (
            "Here are the commands you can use:\n"
            "/start - Start the bot\n"
            "/help - Get Help Information\n"
            "/info - Get Information about the Bot\n"
        )
        await event.respond(help_text)
        logging.info(f"Help command received from {event.sender_id}")

    # معالج الاستجابة للكلمات الرئيسية
    @client.on(events.NewMessage)
    async def keyword_responder(event):
        message = event.text.lower()
        if message in ['/start', '/help', '/info']:
            return

        # استجابة من العميل AI باستخدام OpenRouter
        try:
            completion = openrouter_client.chat.completions.create(
                extra_headers={
                    "HTTP-Referer": "<YOUR_SITE_URL>",  # يمكن استبدال الرابط بموقعك
                    "X-Title": "<YOUR_SITE_NAME>",  # يمكن استبدال اسم الموقع
                },
                extra_body={},
                model="google/gemini-2.5-pro-exp-03-25:free",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": message
                            }
                        ]
                    }
                ]
            )
            # إرسال الاستجابة
            response = completion.choices[0].message.content
            await event.respond(response)
            logging.info(f"Message received from {event.sender_id}: {event.text}")
        except Exception as e:
            logging.error(f"Error occurred while fetching response: {e}")
            await event.respond("Sorry, I couldn't process your request at the moment.")

    # تشغيل العميل حتى يتم قطع الاتصال
    await client.run_until_disconnected()

# تشغيل الدالة الرئيسية
await main()